In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geojsonio
import config

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
ge = pd.read_csv("data/election-data-2004-2018.csv")

In [3]:
ge.head(3).T

,0,1,2
year,2004,2004,2004
state,Perlis,Perlis,Perlis
seat_code_no,1,1,2
seat_code,P1,P1,P2
seat_name,Padang Besar,Padang Besar,Kangar
candidate,Datuk Seri Azmi Khalid,Wan Kharizal Wan Khazim,Datuk Radzi Sheikh Ahmad
party,UMNO,PAS,UMNO
coalition,BN,Pakatan,BN
winlose,1,0,1
is_uncontested,0,0,0


1. Get only results from GE14
2. Get only winners
3. Create winner JSONs for each party

In [4]:
ge11 = ge[ge["year"] == 2004]
ge12 = ge[ge["year"] == 2008]
ge13 = ge[ge["year"] == 2013]
ge14 = ge[ge["year"] == 2018]

In [5]:
for year in ge["year"].unique():
    ge_year = ge[ge["year"] == year]
    parties = ge_year["party"].unique()
    print("Competing parties in year {}:\n{}\n".format(year, ", ".join(parties)))

Competing parties in year 2004:
UMNO, PAS, MCA, PKR, Ind, DAP, GERAKAN, PPP, MIC, MDP, IND, PBS, SETIA, UPKO, SAPP, PBRS, LDP, PASOK, SPDP, SNAP, PBB, SUPP, STAR, PBDS

Competing parties in year 2008:
UMNO, PAS, PKR, MCA, IND, DAP, GERAKAN, PPP, MIC, PRM, PBS, BERSEKUTU, UPKO, SAPP, PASOK, PBRS, LDP, SPDP, SNAP, PBB, SUPP, PRS

Competing parties in year 2013:
PAS, UMNO, IND, PKR, MCA, BERJASA, BERSAMA, KITA, DAP, PCM, MIC, GERAKAN, PPP, SAPP, STAR, PBS, UPKO, PBRS, LDP, SPDP, PBB, SUPP, PRS, SWP

Competing parties in year 2018:
UMNO, BERSATU, PAS, PKR, AMANAH, MCA, BEBAS, PRM, DAP, PCM, MU, GERAKAN, PFP, MIC, PSM, BERJASA, IKATAN, MYPPP, PAP, WARISAN, HR, PPRS, PBS, PCS, UPKO, SAPP, ANAKNEGERI, STAR, PBRS, LDP, PDP, PBB, SUPP, PRS, PBDSB, PBK, PEACE



In [6]:
ge14_seats_won = ge14[ge14["winlose"] == 1]

In [7]:
ge14_seats_bn = ge14_seats_won[ge14_seats_won["coalition"] == "BN"]
ge14_seats_ph = ge14_seats_won[ge14_seats_won["coalition"] == "Pakatan"]
ge14_seats_gs = ge14_seats_won[ge14_seats_won["coalition"] == "PAS"]

In [8]:
ge13_seats_won = ge13[ge13["winlose"] == 1]
ge13_seats_bn = ge13_seats_won[ge13_seats_won["coalition"] == "BN"]
ge13_seats_ph = ge13_seats_won[ge13_seats_won["coalition"] == "Pakatan"]
ge13_seats_gs = ge13_seats_won[ge13_seats_won["coalition"] == "PAS"]

ge12_seats_won = ge12[ge12["winlose"] == 1]
ge12_seats_bn  = ge12_seats_won[ge12_seats_won["coalition"] == "BN"]
ge12_seats_ph  = ge12_seats_won[ge12_seats_won["coalition"] == "Pakatan"]
ge12_seats_gs  = ge12_seats_won[ge12_seats_won["coalition"] == "PAS"]

ge11_seats_won = ge11[ge11["winlose"] == 1]
ge11_seats_bn  = ge11_seats_won[ge11_seats_won["coalition"] == "BN"]
ge11_seats_ph  = ge11_seats_won[ge11_seats_won["coalition"] == "Pakatan"]
ge11_seats_gs  = ge11_seats_won[ge11_seats_won["coalition"] == "PAS"]

In [10]:
ge14_geo = gpd.read_file("data/parliamentary-boundaries-2018-working.geojson")

In [11]:
ge14_geo.head()

,KodPAR,Parliament,State,geometry
0,054,GERIK,PERAK,(POLYGON ((100.9933944673964 5.507324842759963...
1,055,LENGGONG,PERAK,(POLYGON ((100.9293339657613 5.514131158215433...
2,056,LARUT,PERAK,(POLYGON ((100.8672078753412 5.428777928590932...
3,058,BAGAN SERAI,PERAK,(POLYGON ((100.6241018705403 5.141014699246791...
4,057,PARIT BUNTAR,PERAK,(POLYGON ((100.4127561987131 4.992960828699954...


In [12]:
ge14_geo["KodPAR"] = pd.to_numeric(ge14_geo["KodPAR"])

In [13]:
ge14_geo_bn = ge14_geo[ge14_geo["KodPAR"].isin(ge14_seats_bn["seat_code_no"])]
ge14_geo_ph = ge14_geo[ge14_geo["KodPAR"].isin(ge14_seats_ph["seat_code_no"])]
ge14_geo_gs = ge14_geo[ge14_geo["KodPAR"].isin(ge14_seats_gs["seat_code_no"])]

In [14]:
ge14_geo_bn.head()

,KodPAR,Parliament,State,geometry
0,54,GERIK,PERAK,(POLYGON ((100.9933944673964 5.507324842759963...
1,55,LENGGONG,PERAK,(POLYGON ((100.9293339657613 5.514131158215433...
2,56,LARUT,PERAK,(POLYGON ((100.8672078753412 5.428777928590932...
3,58,BAGAN SERAI,PERAK,(POLYGON ((100.6241018705403 5.141014699246791...
6,61,PADANG RENGAS,PERAK,"(POLYGON ((100.8067462782922 4.85724527156128,..."


In [15]:
ge13_geo_bn = ge14_geo[ge14_geo["KodPAR"].isin(ge13_seats_bn["seat_code_no"])]
ge13_geo_ph = ge14_geo[ge14_geo["KodPAR"].isin(ge13_seats_ph["seat_code_no"])]
ge13_geo_gs = ge14_geo[ge14_geo["KodPAR"].isin(ge13_seats_gs["seat_code_no"])]

ge12_geo_bn = ge14_geo[ge14_geo["KodPAR"].isin(ge12_seats_bn["seat_code_no"])]
ge12_geo_ph = ge14_geo[ge14_geo["KodPAR"].isin(ge12_seats_ph["seat_code_no"])]
ge12_geo_gs = ge14_geo[ge14_geo["KodPAR"].isin(ge12_seats_gs["seat_code_no"])]

ge11_geo_bn = ge14_geo[ge14_geo["KodPAR"].isin(ge11_seats_bn["seat_code_no"])]
ge11_geo_ph = ge14_geo[ge14_geo["KodPAR"].isin(ge11_seats_ph["seat_code_no"])]
ge11_geo_gs = ge14_geo[ge14_geo["KodPAR"].isin(ge11_seats_gs["seat_code_no"])]

In [16]:
with open ("data/ge14-geo-bn.geojson", "w") as f:
    f.write(ge14_geo_bn.to_json())
    
with open ("data/ge14-geo-ph.geojson", "w") as f:
    f.write(ge14_geo_ph.to_json())
    
with open ("data/ge14-geo-gs.geojson", "w") as f:
    f.write(ge14_geo_gs.to_json())
    
with open ("data/ge13-geo-bn.geojson", "w") as f:
    f.write(ge13_geo_bn.to_json())
    
with open ("data/ge13-geo-ph.geojson", "w") as f:
    f.write(ge13_geo_ph.to_json())
    
with open ("data/ge13-geo-gs.geojson", "w") as f:
    f.write(ge13_geo_gs.to_json())
    
with open ("data/ge12-geo-bn.geojson", "w") as f:
    f.write(ge12_geo_bn.to_json())
    
with open ("data/ge12-geo-ph.geojson", "w") as f:
    f.write(ge12_geo_ph.to_json())
    
with open ("data/ge12-geo-gs.geojson", "w") as f:
    f.write(ge12_geo_gs.to_json())
    
with open ("data/ge11-geo-bn.geojson", "w") as f:
    f.write(ge11_geo_bn.to_json())
    
with open ("data/ge11-geo-ph.geojson", "w") as f:
    f.write(ge11_geo_ph.to_json())
    
with open ("data/ge11-geo-gs.geojson", "w") as f:
    f.write(ge11_geo_gs.to_json())